In [1]:
import yt
import numpy as np
from astropy.table import QTable
import matplotlib.pyplot as plt

In [2]:
G = yt.units.gravitational_constant
sqrt2 = 2.0**0.5

In [3]:
# This defines a hot gas filter, which creates a new particle type "hot_gas" from the "gas" type, 
# Gas which counts has density < 3.0e-25 g/cc and temperature > 5.0e5 K

def hot_gas(pfilter, data):
    # Here, pfilter.filtered_type is "gas", set by the call to yt.add_particle_filter below
    pfilter1 = data[pfilter.filtered_type, "density"] < 3.0e-25
    pfilter2 = data[pfilter.filtered_type, "temperature"] > 5.0e5
    return pfilter1 & pfilter2

# This lets yt know about the particle filter
yt.add_particle_filter(
    "hot_gas",
    function=hot_gas,
    filtered_type="gas",
    requires=["density", "temperature"],
)
def _density_squared(field, data):
    mp = 1.0*data.ds.units.mp
    n = data["gas","density"]/mp	
    return n*n


In [4]:
halo_ids = np.arange(10)
print(halo_ids)

[0 1 2 3 4 5 6 7 8 9]


In [5]:
fns = [f"halos/halo_{i}.hdf5" for i in halo_ids]
print(fns)

['halos/halo_0.hdf5', 'halos/halo_1.hdf5', 'halos/halo_2.hdf5', 'halos/halo_3.hdf5', 'halos/halo_4.hdf5', 'halos/halo_5.hdf5', 'halos/halo_6.hdf5', 'halos/halo_7.hdf5', 'halos/halo_8.hdf5', 'halos/halo_9.hdf5']


In [6]:
t = QTable.read("clusters.dat", format="ascii.commented_header")

# keep only rows whose id is in halo_ids
mask = np.isin(t["id"], halo_ids)
t_sub = t[mask]

print(len(t_sub))   # should be 10 now

10


In [7]:
t_sub

id,M200c,r200c,M500c,r500c,x,y,z
int64,float64,float64,float64,float64,float64,float64,float64
0,1535789976518656.0,2432.72216796875,1155069043015680.0,1630.108154296875,43718.8125,48813.640625,147594.953125
1,1307340129173504.0,2305.65966796875,964268006572032.0,1534.8707275390625,81856.5078125,121018.3203125,194532.6875
2,1033314370584576.0,2131.685302734375,703479337189376.0,1381.759033203125,119228.15625,67811.4921875,196039.390625
3,899670222045184.0,2035.5552978515625,631160879185920.0,1332.6409912109375,86512.828125,81893.5625,52624.5546875
4,841819696922624.0,1990.9473876953125,599063716167680.0,1309.6837158203125,134485.390625,36361.37890625,35270.6171875
5,733527398154240.0,1901.6170654296875,391693769637888.0,1136.7021484375,46909.2734375,73091.875,143122.296875
6,459285179924480.0,1626.8896484375,303051516674048.0,1043.526611328125,172315.078125,111833.109375,163072.765625
7,581220677189632.0,1759.701904296875,378314778738688.0,1123.628662109375,197691.921875,112083.5234375,198472.453125
8,398251614273536.0,1551.316162109375,295166325817344.0,1034.408447265625,48910.67578125,189379.3125,98636.609375


In [8]:
dss = yt.DatasetSeries(fns)
print(dss)

In [ ]:
lambdas = []
for halo_id, ds in zip(halo_ids, dss):
    ds.add_field(
        ("gas", "density_squared"),
        _density_squared,
        units="cm**-6",
        sampling_type="local",
    )
    ds.add_particle_filter("hot_gas")  # add the hot_gas filter to the dataset
    # find the center, which we're defining as the DM particle with the minimum potential.
    # the function returns two values, the first is the value of the minimum potential and the
    # second is the location of it. the "_" symbol is Python-speak for "throw this value away
    # since we don't care about it"
    _, c = ds.find_min(("PartType1", "Potential"))
    # Find r500c and M500c from the table for this halo
    row = t[t["id"] == halo_id][0]
    r500c = row["r500c"]*yt.units.kpc
    M500c = row["M500c"]*yt.units.Msun
    # Create a sphere object, centered on the position we found, with some radius
    sp = ds.sphere(c, r500c)
    # Find the total angular momentum vector in this sphere and normalize it.
    L = yt.YTArray(
        [
            sp["hot_gas", "angular_momentum_x"].sum(),
            sp["hot_gas", "angular_momentum_y"].sum(),
            sp["hot_gas", "angular_momentum_z"].sum(),
        ]
    )
    print(L)
    L500c = np.sqrt(np.dot(L, L))
    v500c = np.sqrt(G*M500c/r500c)
    print(v500c) 
    lambdas.append(L500c/(sqrt2*M500c*r500c*v500c))
    L /= L500c
    # L is now the normalized angular momentum vector, 
    # we can set it as the "normal" vector for cylindrical coordinates
    # in the sphere by using `set_field_parameter`
    sp.set_field_parameter("normal", L)
    # This is a 1D profile. We are profiling only on radius.
    units = {
        ("hot_gas", "radius"): "kpc",
        ("hot_gas", "velocity_spherical_theta"): "km/s",
        ("hot_gas", "velocity_spherical_radius"): "km/s",
        ("hot_gas", "velocity_spherical_phi"): "km/s",
        ("hot_gas", "velocity_cylindrical_theta"): "km/s",
        ("hot_gas", "velocity_cylindrical_radius"): "km/s",
        ("hot_gas", "sound_speed"): "km/s",
    }
    logs = {
        ("hot_gas", "radius"): False,
        ("hot_gas", "velocity_spherical_theta"): False,
        ("hot_gas", "velocity_spherical_phi"): False,
        ("hot_gas", "velocity_cylindrical_theta"): False,
        ("hot_gas", "sound_speed"): False,
        ("hot_gas", "velocity_spherical_radius"): False,
        ("hot_gas", "velocity_cylindrical_radius"): False,
    }
    extrema = {("hot_gas", "radius"): (0.0, r500c.v)}
    p1d_sph = sp.profile(
        ("hot_gas", "radius"),
        [
            ("hot_gas", "velocity_spherical_theta"),
            ("hot_gas", "velocity_spherical_radius"),
            ("hot_gas", "sound_speed"),
            ("hot_gas", "velocity_spherical_phi"),
            ("hot_gas", "velocity_cylindrical_theta"),
            ("hot_gas", "velocity_cylindrical_radius"),

        ],
        weight_field=("hot_gas", "mass"),
        units=units,
        logs=logs,
        extrema=extrema,
    )
    # We can convert the profile to an AstroPy table and save it to disk as a FITS table file
    t_sph = p1d_sph.to_astropy_table(only_used=True, include_std=True)
    t_sph.write(f"{str(ds)}_vcurve_sph.fits", overwrite=True)
    
t_sub["lambda"] = lambdas
t_sub.write("clusters_w_lambdas.dat", format="ascii.commented_header", overwrite = True)

FileNotFoundError: No such file or directory: 'halos/halo_0.hdf5'.
(Also tried '/Users/jzuhone/Data/yt/test_outputs/halos/halo_0.hdf5').

In [ ]:
t_sub

id,M200c,r200c,M500c,r500c,x,y,z,lambda
int64,float64,float64,float64,float64,float64,float64,float64,float64
0,1535789976518656.0,2432.72216796875,1155069043015680.0,1630.108154296875,43718.8125,48813.640625,147594.953125,0.010348951653748679
1,1307340129173504.0,2305.65966796875,964268006572032.0,1534.8707275390625,81856.5078125,121018.3203125,194532.6875,0.002450525800981862
2,1033314370584576.0,2131.685302734375,703479337189376.0,1381.759033203125,119228.15625,67811.4921875,196039.390625,0.0004162510476473753
3,899670222045184.0,2035.5552978515625,631160879185920.0,1332.6409912109375,86512.828125,81893.5625,52624.5546875,0.006179656311104861
4,841819696922624.0,1990.9473876953125,599063716167680.0,1309.6837158203125,134485.390625,36361.37890625,35270.6171875,0.0022526656501491416
5,733527398154240.0,1901.6170654296875,391693769637888.0,1136.7021484375,46909.2734375,73091.875,143122.296875,0.0033369964340542843
6,459285179924480.0,1626.8896484375,303051516674048.0,1043.526611328125,172315.078125,111833.109375,163072.765625,0.005142900590593633
7,581220677189632.0,1759.701904296875,378314778738688.0,1123.628662109375,197691.921875,112083.5234375,198472.453125,0.006041094402259922
8,398251614273536.0,1551.316162109375,295166325817344.0,1034.408447265625,48910.67578125,189379.3125,98636.609375,0.0036251130447404888
